In [1]:
pip install python-vlc

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show python-vlc

Name: python-vlc
Version: 3.0.101
Summary: VLC bindings for python.
Home-page: http://wiki.videolan.org/PythonBinding
Author: Olivier Aubert
Author-email: contact@olivieraubert.net
License: GPL
Location: c:\users\myhom\anaconda3\lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import queue
import platform

from PyQt5 import QtWidgets, QtGui, QtCore
import vlc
class MiniPlayer(QtWidgets.QMainWindow):

    def __init__(self, data_queue, master=None):
        QtWidgets.QMainWindow.__init__(self, master)
        self.setWindowTitle("Mini Player")
        self.statusbar = self.statusBar()
        self.statusbar.showMessage("Ready")
        
        # Create a basic vlc instance
        self.instance = vlc.Instance()
        self.media = None
        self.mediaplayer = self.instance.media_player_new()

        self.init_ui()
        self.open_file()
        
        self.setWindowFlags(QtCore.Qt.WindowStaysOnTopHint)
        
        self.timer = QtCore.QTimer(self)
        self.timer.setInterval(10)
        self.timer.timeout.connect(self.update_ui)

        self.data_queue = data_queue
        self.timer.start()
        
    def init_ui(self):
        self.widget = QtWidgets.QWidget(self)
        self.setCentralWidget(self.widget)

        if platform.system() == "Darwin":  # for MacOS
            self.videoframe = QtWidgets.QMacCocoaViewContainer(0)
        else:
            self.videoframe = QtWidgets.QFrame()

        self.palette = self.videoframe.palette()
        self.palette.setColor(QtGui.QPalette.Window, QtGui.QColor(0, 0, 0))
        self.videoframe.setPalette(self.palette)
        self.videoframe.setAutoFillBackground(True)

        self.vboxlayout = QtWidgets.QVBoxLayout()
        self.vboxlayout.addWidget(self.videoframe)
        self.widget.setLayout(self.vboxlayout)

    def open_file(self):
        
        filename =  "저장 경로 템플릿.f248.webm"
        # getOpenFileName returns a tuple, so use only the actual file name
        self.media = self.instance.media_new(filename)

        self.mediaplayer.set_media(self.media)

        # Parse the metadata of the file
        self.media.parse()
        
        # Set the title of the track as the window title
        self.setWindowTitle("{}".format(self.media.get_meta(0)))

        if platform.system() == "Linux":  # for Linux using the X Server
            self.mediaplayer.set_xwindow(int(self.videoframe.winId()))
        elif platform.system() == "Windows":  # for Windows
            self.mediaplayer.set_hwnd(int(self.videoframe.winId()))
        elif platform.system() == "Darwin":  # for MacOS
            self.mediaplayer.set_nsobject(int(self.videoframe.winId()))

        self.mediaplayer.play()

    def update_ui(self):
        self.update_statusbar()
        if self.mediaplayer.is_playing()== 0:
            self.mediaplayer.stop()
            self.mediaplayer.set_position(0.0)
            self.mediaplayer.play()

        

    def update_statusbar(self):
        mtime = QtCore.QTime(0, 0, 0, 0)
        time = mtime.addMSecs(self.mediaplayer.get_time())
        self.statusbar.showMessage(time.toString())


def main():
    app = QtWidgets.QApplication(sys.argv)

    data_queue = queue.Queue()

    player = MiniPlayer(data_queue)
    player.show()
    player.resize(480, 480)
    sys.exit(app.exec_())
    
if __name__ == "__main__":
    main()


SystemExit: 0

C:\Users\myhom\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
